In [27]:
!pip install torch torchvision transformers matplotlib seaborn scikit-learn pillow


Defaulting to user installation because normal site-packages is not writeable


In [28]:
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from PIL import Image
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Diretórios
base_dir = "D:/Mamografias Coenc/Abril"
images_dir = os.path.join(base_dir, "images")
reports_dir = os.path.join(base_dir, "reports")

# Parâmetros
BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 0.001

# Tokenizer para laudos
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [29]:
class MamografiaDataset(Dataset):
    def __init__(self, images_dir, reports_dir, transform=None, tokenizer=None):
        self.images_dir = images_dir
        self.reports_dir = reports_dir
        self.transform = transform
        self.tokenizer = tokenizer
        
        self.images = sorted(os.listdir(images_dir))
        self.reports = sorted(os.listdir(reports_dir))
        
        print(f"Total de imagens: {len(self.images)}")
        print(f"Total de relatórios: {len(self.reports)}")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if idx >= len(self.images) or idx >= len(self.reports):  # Verificação de índice válido
            print(f"⚠️ Índice inválido: {idx}. Tamanho máximo: {len(self.images)-1}")
            return None  # Evita erro ao tentar acessar um índice inexistente
        
        img_path = os.path.join(self.images_dir, self.images[idx])
        txt_path = os.path.join(self.reports_dir, self.reports[idx])
        
        print(f"🔹 Carregando imagem: {img_path}")
        print(f"🔹 Carregando relatório: {txt_path}")
        
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        text = load_text(txt_path)  # Usa a função segura de leitura
        inputs = self.tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")

        label = int("normal" not in text.lower())  # Exemplo de categorização binária (0: Normal, 1: Alterado)

        return image, inputs['input_ids'].squeeze(0), inputs['attention_mask'].squeeze(0), torch.tensor(label)


In [30]:
def load_text(file_path):
    """Lê o arquivo de texto tentando várias codificações."""
    encodings = ['utf-8', 'ISO-8859-1', 'windows-1252']
    
    for enc in encodings:
        try:
            with open(file_path, 'r', encoding=enc) as f:
                return f.read()
        except UnicodeDecodeError:
            continue  # Tenta a próxima codificação

    print(f"⚠️ Erro ao ler {file_path} - Nenhuma codificação funcionou.")
    return ""  # Retorna string vazia se falhar


In [31]:
# Transformação de imagens
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = MamografiaDataset(images_dir, reports_dir, transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

Total de imagens: 215
Total de relatórios: 52


In [32]:
import torch
import torch.nn as nn
import torchvision.models as models
from transformers import BertModel

class MultiModalNet(nn.Module):
    def __init__(self):
        super(MultiModalNet, self).__init__()

        # Modelo CNN (ResNet18)
        self.cnn = models.resnet18(pretrained=True)
        in_features = self.cnn.fc.in_features  # Obtém a dimensão de saída da ResNet
        self.cnn.fc = nn.Linear(in_features, 256)

        # Modelo BERT
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.text_fc = nn.Linear(768, 256)

        # Camada final de classificação
        self.classifier = nn.Linear(256 + 256, 2)

    def forward(self, image, input_ids, attention_mask):
        img_features = self.cnn(image)
        text_features = self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        text_features = self.text_fc(text_features)

        combined = torch.cat((img_features, text_features), dim=1)
        output = self.classifier(combined)
        return output

In [33]:
# Inicializa modelo, otimizador e loss
model = MultiModalNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Treinamento
def train(model, dataloader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, input_ids, attention_mask, labels in dataloader:
            images, input_ids, attention_mask, labels = images.to(device), input_ids.to(device), attention_mask.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images, input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader):.4f}")


In [34]:
# Avaliação
def evaluate(model, dataloader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, input_ids, attention_mask, labels in dataloader:
            images, input_ids, attention_mask, labels = (
                images.to(device), input_ids.to(device), attention_mask.to(device), labels.to(device)
            )
            outputs = model(images, input_ids, attention_mask)
            predictions = torch.argmax(outputs, dim=1).cpu().numpy()
            y_true.extend(labels.cpu().numpy())  # Garante que labels está na CPU antes de converter para NumPy
            y_pred.extend(predictions)
    return y_true, y_pred

# Treinar modelo
train(model, dataloader, criterion, optimizer, EPOCHS)

# Definir diretórios
base_dir = "D:/Mamografias Coenc/Abril"
images_dir = os.path.join(base_dir, "images")
reports_dir = os.path.join(base_dir, "reports")


⚠️ Índice inválido: 62. Tamanho máximo: 214
⚠️ Índice inválido: 203. Tamanho máximo: 214
⚠️ Índice inválido: 155. Tamanho máximo: 214
⚠️ Índice inválido: 120. Tamanho máximo: 214
⚠️ Índice inválido: 84. Tamanho máximo: 214
🔹 Carregando imagem: D:/Mamografias Coenc/Abril\images\32867_32060_3.png
🔹 Carregando relatório: D:/Mamografias Coenc/Abril\reports\33383_32616.pdf


TypeError: 'NoneType' object is not callable

In [ ]:
# Avaliação e matriz de confusão
y_true, y_pred = evaluate(model, dataloader)
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Alterado'], yticklabels=['Normal', 'Alterado'])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()

print("\n", classification_report(y_true, y_pred, target_names=['Normal', 'Alterado']))
